# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os

# Serper api key is free get from website have few free credits
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Seach website
search_tool = SerperDevTool()

# Scrape website
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

- Many companies are already using crewai for risk analysis

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    
    # setting process to operate hirerchically
    # manager kind of approach
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst so we can identify trends and predict market movements effectively.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
"context": "The task involves using statistical modeling and machine learning techniques to analyze market data for AAPL in order to identify trends and predict market movements. The goal is to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Summary
News
Chart
Community
Statistics
Historical Data
Profile
Financials
Analysis
Options
Holders
Sustainability

NasdaqGS - Nasdaq Real Time Price • USD
Apple Inc. (AAPL)
Follow
Compare
186.88
-4.02 (-2.11%)
At close: May 23 at 4:00 PM EDT
187.28
+0.40 (+0.21%)
After hours: May 23 at 7:59 PM EDT
1D
5D
-1.56%
3M
1.36%
6M
-2.32%
YTD
-2.93%
1Y
7.28%
5Y
308.97%
All
145,504.15%

Key Events
Mountain Line Candle Baseline Mountain Bar
Advanced Chart
Loading Chart for AAPL
9/21 12:03 PM DELL
Date
Close
Open
High
Low
Volume
Previous Close
190.90
Open
191.00
Bid
186.78 x 100
Ask
186.86 x 100
Day's Range
186.62 - 191.00
52 Week Range
164.08 - 199.62
Volume
50,481,918
Avg. Volume
63,953,180
Market Cap (intraday)
2.866T
Beta (5Y Monthly)
1.26
PE Ratio (TTM)
29.06
EPS (TTM)
6.43
Earnings Date
Aug 1, 2024 - Aug 5, 2024
Forward Dividend & Yield
1.00 (0.54%)
Ex-Dividend Date
May 10, 2024
1y Target Est
202.83

Apple Inc

Final Answer: This report on Apple Inc. (AAPL) provides insights into the stock price performance, key events, financial highlights, and comparisons to other companies in the consumer electronics industry. It offers a comprehensive overview of AAPL's market data, including valuation measures, profitability, income statement, balance sheet, cash flow, research analysis, and company insights. By continuously monitoring and analyzing this data, we can identify trends and predict market movements, ultimately providing valuable insights and alerts about significant market opportunities or threats for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: This report on Apple Inc. (AAPL) provides insights into the stock price performance, key events, financial highlights, and comparisons to other companies in the consumer electronics industry. It offers a comprehensive overview of AAPL's market data, including valuation measures, profitability, income statement, balance sheet, cash f

The content retrieved from the website does not provide any information related to AAPL's stock price performance, key events, financial highlights, or comparisons to other companies in the consumer electronics industry. 

Final Answer: Unfortunately, the content from the website did not provide the necessary insights on AAPL's stock price performance, key events, financial highlights, and comparisons to other companies in the consumer electronics industry. I will need to search for alternative sources to gather this information.

> Finished chain.
 

Unfortunately, the content from the website did not provide the necessary insights on AAPL's stock price performance, key events, financial highlights, and comparisons to other companies in the consumer electronics industry. I will need to search for alternative sources to gather this information.

Thought: 
I need to find an alternative source to gather the necessary information for developing the best execution plans for AAPL trading st

Final Answer: 

Title: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/
Snippet: AAPL. 323.76%. S&P 500. 84.43%. Compare To: AAPL. Compare. Select to analyze similar companies using key performance metrics; select up to 4 stocks. AAPL. Apple ...

Title: AAPL Intrinsic Valuation and Fundamental Analysis - Apple Inc
Link: https://www.alphaspread.com/security/nasdaq/aapl/summary
Snippet: Apple Inc (NASDAQ:AAPL) Intrinsic Valuation. Check if AAPL is overvalued or undervalued under the bear, base, and bull scenarios of the company's future.

Title: How Does Apple Stock React To Product Releases? - Dividend.com
Link: https://www.dividend.com/how-to-invest/how-does-apple-stock-react-to-product-releases/
Snippet: We took a look at Apple's past product launches - here's how its stock price has reacted.

Title: Consumer Electronics Stock Performance - Yahoo Finance
Link: https://finance.yahoo.com/sectors/technology/consumer-electron

Final Answer: 
AAPL stock price, news, and fundamental analysis from Yahoo Finance:

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
AAPL 186.88 -2.11% At close: May 23 at 4:00 PM EDT
AAPL 187.28 +0.40% After hours: May 23 at 7:59 PM EDT
1D -1.56% 3M 1.36% 6M -2.32% YTD -2.93% 1Y 7.28% 5Y 308.97%

Key Events:
- Mountain Line Candle Baseline Mountain Bar
- Market Cap (intraday): 2.866T
- PE Ratio (TTM): 29.06
- EPS (TTM): 6.43
- Earnings Date: Aug 1, 2024 - Aug 5, 2024
- Forward Dividend & Yield: 1.00 (0.54%)
- Ex-Dividend Date: May 10, 2024

Overview: 
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers various services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay.

Performance Overview:
Trailing total returns as of 5/23/2024:
- YTD Return AAPL: 2.68%
- 1-Year Return AAPL: 7.85%
- 3-Year Return AAPL: 51.54%
- 5-Year Re

Action Input: {"search_query": "potential risks associated with current market trends and financial indicators for AAPL"} 


Search results: Title: Apple (AAPL) Stock Risk Factors - TipRanks.com
Link: https://www.tipranks.com/stocks/aapl/risk-factors
Snippet: Debt & Financing – risks related to debt, funding, financing and interest rates, financial investments. Corporate Activity and Growth – risks related to ...
---
Title: Investing In Apple: The Risks and Rewards - Investopedia
Link: https://www.investopedia.com/stock-analysis/021815/investing-apple-risks-rewards-aapl.aspx
Snippet: The risks lie in the company's future ability to keep those consumers while drawing in new ones in more markets. New technology makes consumers fickle. They ...
---
Title: Understanding Apple stock performance today: (USA Stocks:AAPL)
Link: https://www.macroaxis.com/performance/AAPL/Apple-Inc
Snippet: Apple Inc right now shows a risk of 1.53%. Please confirm Apple Inc sortino ratio, skewness, price action 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Title: Risk Analysis Report for Apple Inc. (AAPL) Trading Strategies

Introduction:
Apple Inc. (AAPL) is a leading technology company known for its innovative products and services. As part of evaluating the risks associated with the proposed trading strategies and execution plans for AAPL, a comprehensive risk analysis report has been prepared based on the gathered information from multiple sources.

Risk Analysis:
1. Market Volatility Risk:
The stock performance data for AAPL indicates fluctuations in the stock price over different time frames. The YTD return is positive, but the 6-month and YTD returns show a decline, suggesting market volatility. This poses a risk to the trading strategies as sudden market movements can impact investment decisions.

2. Competition Risk:
Comparisons to other companies in the consumer electronics industry show that AAPL faces competition from companies like SONY, KOSS, and Xiaomi Corporation. The market share and performance of these competitors can affect AAPL's stock price and market position. Mitigation strategies may involve monitoring competitor activities and staying ahead in innovation.

3. Fundamental Analysis Risk:
The fundamental analysis of AAPL reveals key financial indicators such as PE ratio, EPS, market cap, and earnings date. Any negative financial news or changes in these indicators can impact investor confidence and stock price. Continuous monitoring of fundamental metrics is crucial to mitigate this risk.

4. Product Launch Risk:
Past product launches have influenced AAPL's stock price significantly. As seen in the historical data, product releases can lead to price fluctuations. The reaction of the stock price to new product launches must be considered in the trading strategies. Mitigation strategies may involve diversifying the portfolio to reduce dependency on specific product launches.

5. Macro-Economic Risk:
External factors such as global economic conditions, political events, and regulatory changes can impact AAPL's stock performance. The company's exposure to international markets and supply chain disruptions pose additional risks. Risk mitigation strategies may involve diversification, hedging, and staying informed about macro-economic trends.

Conclusion:
In conclusion, the risk analysis report highlights several potential risks associated with the proposed trading strategies and execution plans for AAPL. By addressing market volatility, competition, fundamental analysis, product launch risks, and macro-economic factors, the team can develop robust mitigation strategies to optimize trading outcomes and minimize potential losses. Ongoing monitoring and adaptation to changing market conditions are essential for successful trading in AAPL stock.